# Separação treino-teste

In [1]:
from loader_and_preprocess_funcs import *

In [2]:
data =load__data()

In [3]:
top15 = ['Neighborhood','House.Style','Overall.Qual','Full.Bath','Year.Remod.Add','Lot.Area','TotRms.AbvGrd','Fireplaces','Heating.QC','Lot.Frontage',
             'Foundation','Gr.Liv.Area','Kitchen.AbvGr','Open.Porch.SF','Roof.Style','SalePrice']

data = choose_feat(data,top15)

In [4]:
data

,Neighborhood,House.Style,Overall.Qual,Full.Bath,Year.Remod.Add,Lot.Area,TotRms.AbvGrd,Fireplaces,Heating.QC,Lot.Frontage,Foundation,Gr.Liv.Area,Kitchen.AbvGr,Open.Porch.SF,Roof.Style,SalePrice
0,NAmes,1Story,6,1,1960,31770,7,2,Fa,141.0,CBlock,1656,1,62,Hip,215000
1,NAmes,1Story,5,1,1961,11622,5,0,TA,80.0,CBlock,896,1,0,Gable,105000
2,NAmes,1Story,6,1,1958,14267,6,0,TA,81.0,CBlock,1329,1,36,Hip,172000
3,NAmes,1Story,7,2,1968,11160,8,2,Ex,93.0,CBlock,2110,1,0,Hip,244000
4,Gilbert,2Story,5,2,1998,13830,6,1,Gd,74.0,PConc,1629,1,34,Gable,189900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,Mitchel,SLvl,6,1,1984,7937,6,0,TA,37.0,CBlock,1003,1,0,Gable,142500
2926,Mitchel,1Story,5,1,1983,8885,5,0,TA,NaN,CBlock,902,1,0,Gable,131000
2927,Mitchel,SFoyer,5,1,1992,10441,6,0,TA,62.0,PConc,970,1,32,Gable,132000
2928,Mitchel,1Story,5,1,1975,10010,6,1,Gd,77.0,CBlock,1389,1,38,Gable,170000


In [5]:
data = preprocessor(data)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2273 entries, 1 to 2929
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Neighborhood       2273 non-null   object 
 1   House.Style        2273 non-null   object 
 2   Overall.Qual       2273 non-null   object 
 3   Full.Bath          2273 non-null   object 
 4   Year.Remod.Add     2273 non-null   int64  
 5   TotRms.AbvGrd      2273 non-null   object 
 6   Fireplaces         2273 non-null   object 
 7   Heating.QC         2273 non-null   object 
 8   Foundation         2273 non-null   object 
 9   Kitchen.AbvGr      2273 non-null   object 
 10  Roof.Style         2273 non-null   object 
 11  Lot.Area_log       2273 non-null   float64
 12  Lot.Frontage_log   1878 non-null   float64
 13  Gr.Liv.Area_log    2273 non-null   float64
 14  SalePrice_log      2273 non-null   float64
 15  Open.Porch.SF_bin  2273 non-null   object 
dtypes: float64(4), int64(1), obje

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
y = data['SalePrice_log']
X = data.drop(columns=['SalePrice_log'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline Global

In [163]:

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder,PolynomialFeatures
from sklearn.model_selection import cross_val_score
import optuna
from optuna.visualization import *
import plotly
import optuna
import nbformat 

In [67]:
num_cols =['Gr.Liv.Area_log','Lot.Area_log','Lot.Frontage_log','Year.Remod.Add']
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
        ('poly', PolynomialFeatures(degree=3, include_bias=False))
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot',  OneHotEncoder(handle_unknown='ignore', sparse_output=False,drop='first')),
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline,      num_cols),
    ('cat', categorical_pipeline,  cat_cols),
],
remainder='passthrough')



# Modelos

ElasticNet, SVM, Trees e ensamble(cpa)

In [11]:
def print_cv_results(model_name, cv_results):
    cv_results = -cv_results

    print(f'{model_name} CV results:')
    print(cv_results)

    mean_rmse = cv_results.mean()
    std_rmse = cv_results.std()

    print(f'RMSE: {mean_rmse:.2f} +/- {std_rmse:.2f}')
    print()

In [54]:
result = pd.DataFrame()

# Dummy model

In [154]:
from sklearn.dummy import DummyRegressor

In [294]:
from sklearn.impute          import SimpleImputer
from sklearn.preprocessing  import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.compose        import ColumnTransformer
from sklearn.pipeline       import Pipeline
from sklearn.dummy          import DummyRegressor
from sklearn.model_selection import cross_val_score

def objective(trial):
    strategy     = trial.suggest_categorical(
        'strategy',
        ['mean', 'median', 'constant']
    )
    if strategy == 'constant':
        constant_val = trial.suggest_float('constant_val', 0.0, 1.0)
    use_scaler  = trial.suggest_categorical('use_scaler', [False, True])
    if use_scaler:
        with_mean = trial.suggest_categorical('with_mean', [True, False])
        with_std  = trial.suggest_categorical('with_std',  [True, False])
    else:
        with_mean = False
        with_std  = False

    poly_degree = trial.suggest_int('poly_degree', 1, 5)
    num_steps = [('imputer', SimpleImputer(strategy='mean'))]
    if use_scaler:
        num_steps.append((
            'scaler',
            StandardScaler(with_mean=with_mean, with_std=with_std)
        ))
    num_steps.append((
        'poly',
        PolynomialFeatures(degree=poly_degree, include_bias=False)
    ))
    numeric_pipeline = Pipeline(num_steps)

    categorical_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot',  OneHotEncoder(handle_unknown='ignore',
                                  sparse_output=False,
                                  drop='first')),
    ])

    preprocessor = ColumnTransformer(
        [
            ('num', numeric_pipeline, num_cols),
            ('cat', categorical_pipeline, cat_cols),
        ],
        remainder='passthrough'
    )

    dummy_kwargs = {'strategy': strategy}
    if strategy == 'constant':
        dummy_kwargs['constant'] = constant_val

    model = Pipeline([
        ('preproc', preprocessor),
        ('dummy',  DummyRegressor(**dummy_kwargs)),
    ])

    neg_mse = cross_val_score(
        model,
        X_train, y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    ).mean()

    return neg_mse


In [295]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=42)) 

[I 2025-05-03 19:43:13,757] A new study created in memory with name: no-name-d4033223-6c19-4741-9ac8-b89f20ab0b8a


In [296]:
study.optimize(objective, n_trials=100)

[I 2025-05-03 19:43:15,394] Trial 0 finished with value: -0.024529403559009882 and parameters: {'strategy': 'median', 'use_scaler': False, 'poly_degree': 1}. Best is trial 0 with value: -0.024529403559009882.
[I 2025-05-03 19:43:15,500] Trial 1 finished with value: -0.024529403559009882 and parameters: {'strategy': 'median', 'use_scaler': False, 'poly_degree': 5}. Best is trial 0 with value: -0.024529403559009882.
[I 2025-05-03 19:43:15,635] Trial 2 finished with value: -0.024290639236509086 and parameters: {'strategy': 'mean', 'use_scaler': True, 'with_mean': True, 'with_std': False, 'poly_degree': 1}. Best is trial 2 with value: -0.024290639236509086.
[I 2025-05-03 19:43:15,678] Trial 3 finished with value: -19.97385203085658 and parameters: {'strategy': 'constant', 'constant_val': 0.7851759613930136, 'use_scaler': True, 'with_mean': True, 'with_std': True, 'poly_degree': 1}. Best is trial 2 with value: -0.024290639236509086.
[I 2025-05-03 19:43:15,736] Trial 4 finished with value: -

In [297]:
optuna.visualization.plot_optimization_history(study)

In [298]:
best_params = study.best_params 
print(f"Best Hyperparameters: {best_params}")

Best Hyperparameters: {'strategy': 'mean', 'use_scaler': True, 'with_mean': True, 'with_std': False, 'poly_degree': 1}


In [299]:
optuna.visualization.plot_slice(study, params=['strategy','use_scaler','poly_degree','with_std','with_mean'])

In [300]:
optuna.visualization.plot_param_importances(study)

In [301]:
best_strat = best_params['strategy']


In [339]:
numeric_pipeline_dummy = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
        ('poly', PolynomialFeatures(degree=1, include_bias=False)),
        ('scaler', StandardScaler(with_mean=True,with_std=False))
])

categorical_pipeline_dummy = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot',  OneHotEncoder(handle_unknown='ignore', sparse_output=False,drop='first')),
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline_dummy,      num_cols),
    ('cat', categorical_pipeline_dummy,  cat_cols),
],
remainder='passthrough')

dummy = Pipeline([
    ('prepoc',preprocessor),
        ('dummy', DummyRegressor(
            strategy=best_strat
        )),
    
])

In [340]:
dummy_reg_cv = cross_val_score(
        dummy, X_train, y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
print_cv_results('Dummy Regressor',dummy_reg_cv)

Dummy Regressor CV results:
[0.02432599 0.02459076 0.02582598 0.02384896 0.0228615 ]
RMSE: 0.02 +/- 0.00



In [305]:

result['Dummy'] = dummy_reg_cv

# Linear Model 

In [200]:
from sklearn.linear_model import LinearRegression



In [ ]:
def objective(trial):
    fit_intercept = trial.suggest_categorical('fit_intercept', [False, True])
    use_scaler  = trial.suggest_categorical('use_scaler', [False, True])
    if use_scaler:
        with_mean = trial.suggest_categorical('with_mean', [True, False])
        with_std  = trial.suggest_categorical('with_std',  [True, False])
    else:
        with_mean = False
        with_std  = False

    poly_degree = trial.suggest_int('poly_degree', 1, 5)
    num_steps = [('imputer', SimpleImputer(strategy='mean'))]
    if use_scaler:
        num_steps.append((
            'scaler',
            StandardScaler(with_mean=with_mean, with_std=with_std)
        ))
    num_steps.append((
        'poly',
        PolynomialFeatures(degree=poly_degree, include_bias=False)
    ))
    numeric_pipeline = Pipeline(num_steps)

    categorical_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot',  OneHotEncoder(handle_unknown='ignore',
                                  sparse_output=False,
                                  drop='first')),
    ])

    preprocessor = ColumnTransformer(
        [
            ('num', numeric_pipeline, num_cols),
            ('cat', categorical_pipeline, cat_cols),
        ],
        remainder='passthrough'
    )

    model = Pipeline([
        ('preproc', preprocessor),
        ('linear', LinearRegression(fit_intercept=fit_intercept)),
    ])

    neg_mse = cross_val_score(
        model,
        X_train, y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    ).mean()

    return neg_mse

In [283]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=42)) 

[I 2025-05-03 19:39:32,598] A new study created in memory with name: no-name-b3ed6847-d229-40f6-a7f2-5f69718a4a87


In [284]:
study.optimize(objective, n_trials=100)

[I 2025-05-03 19:39:36,712] Trial 0 finished with value: -0.00297418192849084 and parameters: {'fit_intercept': True, 'use_scaler': False, 'poly_degree': 1}. Best is trial 0 with value: -0.00297418192849084.
[I 2025-05-03 19:39:37,321] Trial 1 finished with value: -0.006949901960131823 and parameters: {'fit_intercept': False, 'use_scaler': False, 'poly_degree': 4}. Best is trial 0 with value: -0.00297418192849084.
[I 2025-05-03 19:39:37,390] Trial 2 finished with value: -0.00297418192849084 and parameters: {'fit_intercept': True, 'use_scaler': False, 'poly_degree': 1}. Best is trial 0 with value: -0.00297418192849084.
[I 2025-05-03 19:39:37,670] Trial 3 finished with value: -0.002900251376377368 and parameters: {'fit_intercept': True, 'use_scaler': False, 'poly_degree': 2}. Best is trial 3 with value: -0.002900251376377368.
[I 2025-05-03 19:39:38,507] Trial 4 finished with value: -0.0029234658241886298 and parameters: {'fit_intercept': False, 'use_scaler': True, 'with_mean': False, 'wi

In [285]:
optuna.visualization.plot_optimization_history(study)

In [286]:
best_params = study.best_params 
print(f"Best Hyperparameters: {best_params}")

Best Hyperparameters: {'fit_intercept': False, 'use_scaler': False, 'poly_degree': 2}


In [ ]:
optuna.visualization.plot_slice(study, params=['fit_intercept','use_scaler','poly_degree'])

In [289]:
optuna.visualization.plot_param_importances(study)

In [290]:
numeric_pipeline_linear = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
])

categorical_pipeline_linear = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot',  OneHotEncoder(handle_unknown='ignore', sparse_output=False,drop='first')),
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline_linear,      num_cols),
    ('cat', categorical_pipeline_linear,  cat_cols),
],
remainder='passthrough')

lin_reg = Pipeline([
    ('prepoc',preprocessor),
        ('linear', LinearRegression(
            fit_intercept=False
        )),
    
])

In [291]:
lin_cv = cross_val_score(
        lin_reg, X_train, y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
print_cv_results('Linear Regressor',lin_cv)

Linear Regressor CV results:
[0.00289211 0.0030398  0.0030198  0.00252607 0.00301286]
RMSE: 0.00 +/- 0.00



In [292]:
result['LinearReg'] = lin_cv

In [293]:
result 

,Dummy,ElsticNet,RandomForest,LinearReg
0,-0.024326,-0.003379,-0.004101,-0.002892
1,-0.024591,-0.003366,-0.003917,-0.003040
2,-0.025826,-0.003497,-0.003651,-0.003020
3,-0.023849,-0.002878,-0.003601,-0.002526
4,-0.022861,-0.003048,-0.003593,-0.003013


Rmse de 0.05 (em escala log)

# Elastic Net Regressor

In [17]:
import optuna

/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [164]:
from sklearn.linear_model import ElasticNet

In [306]:
def objective(trial):
    alpha         = trial.suggest_float('alpha',      1e-3, 100.0, log=True)
    l1_ratio      = trial.suggest_float('l1_ratio',   0.1,   1.0)
    fit_intercept = trial.suggest_categorical('fit_intercept', [False, True])
    max_iter      = trial.suggest_int('max_iter',     1000,  5000)

    use_scaler  = trial.suggest_categorical('use_scaler', [False, True])
    if use_scaler:
        with_mean = trial.suggest_categorical('with_mean', [True, False])
        with_std  = trial.suggest_categorical('with_std',  [True, False])
    else:
        with_mean = False
        with_std  = False

    poly_degree = trial.suggest_int('poly_degree', 1, 5)
    num_steps = [('imputer', SimpleImputer(strategy='mean'))]
    if use_scaler:
        num_steps.append((
            'scaler',
            StandardScaler(with_mean=with_mean, with_std=with_std)
        ))
    num_steps.append((
        'poly',
        PolynomialFeatures(degree=poly_degree, include_bias=False)
    ))
            
    numeric_pipeline = Pipeline(num_steps)

    categorical_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot',  OneHotEncoder(handle_unknown='ignore',
                                  sparse_output=False,
                                  drop='first')),
    ])

    preprocessor = ColumnTransformer(
        [
            ('num', numeric_pipeline,      num_cols),
            ('cat', categorical_pipeline,  cat_cols),
        ],
        remainder='passthrough'
    )

    model = Pipeline([
        ('preproc', preprocessor),
        ('est', ElasticNet(
            alpha=alpha,
            l1_ratio=l1_ratio,
            fit_intercept=fit_intercept,
            max_iter=max_iter,
            random_state=42
        )),
    ])

    neg_mse = cross_val_score(
        model, X_train, y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    ).mean()

    return neg_mse



In [307]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=42)) 

[I 2025-05-03 19:45:15,906] A new study created in memory with name: no-name-2a75ef1c-5e1b-49cf-9750-4417d20d60d0


In [308]:
study.optimize(objective, n_trials=100)

/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.034e+00, tolerance: 4.013e+00
  model = cd_fast.enet_coordinate_descent(
/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.228e+00, tolerance: 4.012e+00
  model = cd_fast.enet_coordinate_descent(
/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularis

In [309]:
optuna.visualization.plot_optimization_history(study)

In [310]:
best_params = study.best_params 
print(f"Best Hyperparameters: {best_params}")

Best Hyperparameters: {'alpha': 0.001139641416462081, 'l1_ratio': 0.5217945777947136, 'fit_intercept': True, 'max_iter': 1470, 'use_scaler': True, 'with_mean': False, 'with_std': True, 'poly_degree': 5}


In [311]:
optuna.visualization.plot_slice(study, params=['alpha', 'l1_ratio', 'fit_intercept', 'max_iter','use_scaler','poly_degree','with_mean','with_std'])

In [312]:
optuna.visualization.plot_param_importances(study)

In [313]:
best_alpha = best_params['alpha']
best_l1_ratio = best_params['l1_ratio']
best_fit_intercept = best_params['fit_intercept']
best_max_iter = best_params['max_iter']




In [336]:
numeric_pipeline_elastic = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
        ('poly', PolynomialFeatures(degree=5, include_bias=False)),
        ('scaler',StandardScaler(with_mean=False))
])

categorical_pipeline_elastic = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot',  OneHotEncoder(handle_unknown='ignore', sparse_output=False,drop='first')),
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline_elastic,      num_cols),
    ('cat', categorical_pipeline_elastic,  cat_cols),
],
remainder='passthrough')

elastic = Pipeline([
    ('prepoc',preprocessor),
        ('est', ElasticNet(
            alpha=best_alpha,
            l1_ratio=best_l1_ratio,
            fit_intercept=best_fit_intercept,
            max_iter=best_max_iter,
            random_state=42
        )),
    
])

In [337]:
est_cv = cross_val_score(
        elastic, X_train, y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
print_cv_results('Elastic Net Regressor',est_cv)

Elastic Net Regressor CV results:
[0.00345808 0.00345629 0.00348434 0.00292274 0.00306042]
RMSE: 0.00 +/- 0.00



In [338]:
result['ElsticNet'] = est_cv

# Random Forest Regressor

In [90]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
def objective(trial): 
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 10, 50) 
    min_samples_split = trial.suggest_int('min_samples_split', 2, 32) 
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 32) 

    num_steps = [
        ('imputer', SimpleImputer(strategy='mean'))
    ]
    poly_degree   = trial.suggest_int('poly_degree',1,3)

    num_steps = [('imputer', SimpleImputer(strategy='mean'))]
    
    num_steps.append(('poly',
                      PolynomialFeatures(degree=poly_degree,
                                         include_bias=False)))
            
    numeric_pipeline = Pipeline(num_steps)

    categorical_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot',  OneHotEncoder(handle_unknown='ignore',
                                  sparse_output=False,
                                  drop='first')),
    ])

    preprocessor = ColumnTransformer(
        [
            ('num', numeric_pipeline,      num_cols),
            ('cat', categorical_pipeline,  cat_cols),
        ],
        remainder='passthrough'
    )
    model = Pipeline([
        ('preproc', preprocessor),
        ('randomforest', RandomForestRegressor(
        n_estimators=n_estimators, 
        max_depth=max_depth, 
        min_samples_split=min_samples_split, 
        min_samples_leaf=min_samples_leaf,
        random_state=42
        )),
    ])

    neg_mse = cross_val_score(
        model, X_train, y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    ).mean()

    return neg_mse

In [102]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=42)) 

[I 2025-05-03 15:45:31,619] A new study created in memory with name: no-name-e14df4ff-2051-4e62-8bcb-f2c630aa1740


In [103]:
study.optimize(objective, n_trials=100)

[I 2025-05-03 15:45:40,152] Trial 0 finished with value: -0.004594551395352957 and parameters: {'n_estimators': 437, 'max_depth': 48, 'min_samples_split': 24, 'min_samples_leaf': 20, 'poly_degree': 1}. Best is trial 0 with value: -0.004594551395352957.
[I 2025-05-03 15:45:53,958] Trial 1 finished with value: -0.00445005723682228 and parameters: {'n_estimators': 240, 'max_depth': 12, 'min_samples_split': 28, 'min_samples_leaf': 20, 'poly_degree': 3}. Best is trial 1 with value: -0.00445005723682228.
[I 2025-05-03 15:45:55,329] Trial 2 finished with value: -0.004058536747106786 and parameters: {'n_estimators': 118, 'max_depth': 49, 'min_samples_split': 27, 'min_samples_leaf': 7, 'poly_degree': 1}. Best is trial 2 with value: -0.004058536747106786.
[I 2025-05-03 15:45:58,873] Trial 3 finished with value: -0.0043284876421504324 and parameters: {'n_estimators': 265, 'max_depth': 22, 'min_samples_split': 18, 'min_samples_leaf': 14, 'poly_degree': 1}. Best is trial 2 with value: -0.0040585367

In [104]:
optuna.visualization.plot_optimization_history(study)

In [105]:
best_params = study.best_params 
print(f"Best Hyperparameters: {best_params}")

Best Hyperparameters: {'n_estimators': 930, 'max_depth': 13, 'min_samples_split': 8, 'min_samples_leaf': 2, 'poly_degree': 1}


In [106]:
optuna.visualization.plot_slice(study, params=['n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf','poly_degree'])

In [107]:
optuna.visualization.plot_param_importances(study)

In [108]:
best_n= best_params['n_estimators']
best_max_depth = best_params['max_depth']
best_min_samples_split = best_params['min_samples_split']
best_min_samples_leaf = best_params['min_samples_leaf']

In [111]:
numeric_pipeline_random_tree = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
        ('poly', PolynomialFeatures(degree=1, include_bias=False))
])

categorical_pipeline_elastic = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot',  OneHotEncoder(handle_unknown='ignore', sparse_output=False,drop='first')),
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline_elastic,      num_cols),
    ('cat', categorical_pipeline_elastic,  cat_cols),
],
remainder='passthrough')

rforest = Pipeline([
    ('prepoc',preprocessor),
    ('randomforest', RandomForestRegressor(
        n_estimators=best_n, 
        max_depth=best_max_depth, 
        min_samples_split=best_min_samples_split, 
        min_samples_leaf=best_min_samples_leaf,
        random_state=42
        )),
    
])

In [112]:
rforest_cv = cross_val_score(
        rforest, X_train, y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )

In [113]:
print_cv_results('Random Forest Regressor',rforest_cv)

Random Forest Regressor CV results:
[0.004101   0.0039174  0.003651   0.00360113 0.00359323]
RMSE: 0.00 +/- 0.00



In [114]:
result['RandomForest'] = rforest_cv

# SVR

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler 

In [ ]:
def objective(trial):
    # Hiperparâmetros do SVR com ranges ampliados
    C = trial.suggest_float('C', 1e-5, 1000.0, log=True)
    epsilon = trial.suggest_float('epsilon', 1e-5, 0.5, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
    
    # Parâmetros específicos por kernel
    if kernel == 'poly':
        degree = trial.suggest_int('degree', 2, 5)
        coef0 = trial.suggest_float('coef0', 0.0, 10.0)
    else:
        degree = 3
        coef0 = 1.0
        
    if kernel in ['rbf', 'poly', 'sigmoid']:
        gamma = trial.suggest_float('gamma', 1e-6, 10.0, log=True)
    else:
        gamma = 'scale'
        
    max_iter = trial.suggest_int('max_iter', 1000, 20000, log=True)
    
    # Escolha de escalonador
    scaler_type = trial.suggest_categorical('scaler', ['standard', 'minmax', 'robust'])
    
    # Opções para PolynomialFeatures
    use_poly = trial.suggest_categorical('use_poly', [True, False])
    if use_poly:
        poly_degree = trial.suggest_int('poly_degree', 1, 3)
    else:
        poly_degree = 1
    
    # Pipeline para dados numéricos
    num_steps = [('imputer', SimpleImputer(strategy='mean'))]
    
    if scaler_type == 'standard':
        num_steps.append(('scaler', StandardScaler()))
    elif scaler_type == 'minmax':
        num_steps.append(('scaler', MinMaxScaler()))
    else:
        num_steps.append(('scaler', RobustScaler()))
    
    if use_poly:
        num_steps.append(('poly', PolynomialFeatures(degree=poly_degree, include_bias=False)))
    
    numeric_pipeline = Pipeline(num_steps)

    # Pipeline para dados categóricos
    categorical_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, drop='first')),
    ])

    preprocessor = ColumnTransformer(
        [
            ('num', numeric_pipeline, num_cols),
            ('cat', categorical_pipeline, cat_cols),
        ],
        remainder='passthrough'
    )
    
    # Configuração do SVR com todos os parâmetros
    svr_params = {
        'C': C,
        'epsilon': epsilon,
        'kernel': kernel,
        'max_iter': max_iter
    }
    
    if kernel == 'poly':
        svr_params['degree'] = degree
        svr_params['coef0'] = coef0
    
    if kernel in ['rbf', 'poly', 'sigmoid']:
        svr_params['gamma'] = gamma
    
    model = Pipeline([
        ('preproc', preprocessor),
        ('svr', SVR(**svr_params)),
    ])

    # Avaliação com múltiplas métricas
    score = cross_val_score(
        model, X_train, y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    ).mean()
    
    
    return score

In [367]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=42)) 

[I 2025-05-03 20:24:03,717] A new study created in memory with name: no-name-9dd3e5c8-fd85-4db3-b1bf-0ecb0742a005


In [368]:
study.optimize(objective, n_trials=100)

[I 2025-05-03 20:24:04,596] Trial 0 finished with value: -0.009187839725991067 and parameters: {'C': 0.00991564456663839, 'epsilon': 0.2933445395788398, 'kernel': 'linear', 'max_iter': 1190, 'scaler': 'standard', 'use_poly': False}. Best is trial 0 with value: -0.009187839725991067.
/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=2392).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=2392).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=2392).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/jgfaraco/mini

In [369]:
optuna.visualization.plot_optimization_history(study)

In [370]:
best_params = study.best_params 
print(f"Best Hyperparameters: {best_params}")

Best Hyperparameters: {'C': 0.12323645327131416, 'epsilon': 0.002273055424771213, 'kernel': 'linear', 'max_iter': 14385, 'scaler': 'minmax', 'use_poly': True, 'poly_degree': 2}


In [372]:
optuna.visualization.plot_slice(study, params=['C','epsilon','kernel','gamma','max_iter','poly_degree', 'scaler'])

In [373]:
optuna.visualization.plot_param_importances(study)

In [376]:
best_C = best_params['C']
best_epsilon = best_params['epsilon']
best_kernel = best_params['kernel']
best_max_iter = best_params['max_iter']



In [377]:
numeric_pipeline_SVR = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
        ('scaler',MinMaxScaler())
])

categorical_pipeline_SVR = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot',  OneHotEncoder(handle_unknown='ignore', sparse_output=False,drop='first')),
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline_elastic,      num_cols),
    ('cat', categorical_pipeline_elastic,  cat_cols),
],
remainder='passthrough')

svr = Pipeline([
    ('prepoc',preprocessor),
        ('svr', SVR(
            C=best_C,
            epsilon=best_epsilon,
            kernel=best_kernel,
            max_iter=best_max_iter
        )),
    
])

In [378]:
svr_cv = cross_val_score(
        svr, X_train, y_train,
        cv=5,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
print_cv_results('SVR Regressor',svr_cv)

SVR Regressor CV results:
[0.01023654 0.00424866 0.00452444 0.01362903 0.00501421]
RMSE: 0.01 +/- 0.00



/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=14385).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=14385).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=14385).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=14385).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/Users/jgfaraco/miniconda3/lib/python3.12/site-packages/sklearn/svm/

In [379]:
result['SVR'] = svr_cv

# Avaliando os modelos

In [381]:
result

,Dummy,ElsticNet,RandomForest,LinearReg,SVR
0,-0.024326,-0.003458,-0.004101,-0.002892,-0.010237
1,-0.024591,-0.003456,-0.003917,-0.003040,-0.004249
2,-0.025826,-0.003484,-0.003651,-0.003020,-0.004524
3,-0.023849,-0.002923,-0.003601,-0.002526,-0.013629
4,-0.022861,-0.003060,-0.003593,-0.003013,-0.005014


In [333]:
print('Média de NRMSE do Dummy Model', result['Dummy'].mean())
print('Média de NRMSE do Linear model', result['LinearReg'].mean())
print('Média de NRMSE do ElasticNet model', result['ElsticNet'].mean())
print('Média de NNRMSE do RandomForest model', result['RandomForest'].mean())
print('Média de NNRMSE do SVR model', result['SVR'].mean())

Média de NRMSE do Dummy Model -0.024290639236509086
Média de NRMSE do Linear model -0.002898128376183294
Média de NRMSE do ElasticNet model -0.0032827821041055217
Média de NNRMSE do RandomForest model -0.003772751297930811
Média de NNRMSE do SVR model -0.015095335983458077


In [334]:
from scipy.stats import ttest_rel

In [335]:


scores_dict = result.to_dict(orient='list')
means = {name: np.mean(vals) for name, vals in scores_dict.items()}
best_name = max(means, key=means.get)
print(f"Melhor modelo: {best_name} (erro médio = {means[best_name]:.4f})\n")

alpha = 0.05
n_models = len(scores_dict) - 1
alpha_corr = alpha / n_models
for name, scores in scores_dict.items():
    if name == best_name:
        continue

    best_scores = scores_dict[best_name]
    other_scores = scores
    stat, p = ttest_rel(best_scores, other_scores)

    delta = means[name] - means[best_name]

    signif = 'SIM' if p < alpha_corr else 'NÃO'
    print(
        f"{name:15s} → Δmédia = {delta:+.4f} | "
        f"p-valor = {p:.4f} | Significativo p < {alpha_corr:.4f}? {signif}"
    )


Melhor modelo: LinearReg (erro médio = -0.0029)

Dummy           → Δmédia = -0.0214 | p-valor = 0.0000 | Significativo p < 0.0125? SIM
ElsticNet       → Δmédia = -0.0004 | p-valor = 0.0124 | Significativo p < 0.0125? SIM
RandomForest    → Δmédia = -0.0009 | p-valor = 0.0020 | Significativo p < 0.0125? SIM
SVR             → Δmédia = -0.0122 | p-valor = 0.0000 | Significativo p < 0.0125? SIM


Analisando os coeficientes de variação

In [237]:
pct_errors = (10**result['Dummy'] - 1) * 100

cv = pct_errors.std() / pct_errors.mean() * 100

print(f'CV do modelo Dummy: {-cv:.2f}%')

CV do modelo Dummy: 4.33%


In [1]:
pct_errors = (10**result['LinearReg'] - 1) * 100

cv = pct_errors.std() / pct_errors.mean() * 100

print(f'CV do modelo LinearReg: {-cv:.2f}%')

NameError: name 'result' is not defined

In [239]:
pct_errors = (10**result['RandomForest'] - 1) * 100

cv = pct_errors.std() / pct_errors.mean() * 100

print(f'CV do modelo RandomForest: {-cv:.2f}%')

CV do modelo RandomForest: 5.98%


In [240]:
pct_errors = (10**result['ElsticNet'] - 1) * 100

cv = pct_errors.std() / pct_errors.mean() * 100

print(f'CV do modelo ElasticNet: {-cv:.2f}%')


CV do modelo ElasticNet: 7.99%
